In [ ]:
# This block is to install ase, tk for the GUI, and ipywidgets-jsonschema
import sys

print(sys.executable)
# !{sys.executable} -m pip install ase
# !{sys.executable} -m pip install tk
import tkinter

print(tkinter.Tcl().eval("info patchlevel"))
# !{sys.executable} -m pip install ipywidgets-jsonschema

In [ ]:
from ipywidgets_jsonschema import Form

inputs_schema = {
    "$schema": "http://json-schema.org/draft-07/schema#",
    "type": "object",
    "properties": {
        "slab": {
            "type": "object",
            "properties": {
                "miller:h": {"type": "integer", "default": 1},
                "miller:k": {"type": "integer", "default": 1},
                "miller:l": {"type": "integer", "default": 1},
                "vacuum": {"type": "number", "default": 1},
                "number_of_layers": {"type": "integer", "default": 3},
            },
            "required": ["miller:h", "miller:k", "miller:l", "vacuum", "number_of_layers"],
        },
        "interface": {
            "type": "object",
            "properties": {
                "slab_v:matrix": {
                    "type": "array",
                    "items": {"type": "array", "items": {"type": "integer"}, "minItems": 2, "maxItems": 2},
                    "minItems": 2,
                    "maxItems": 2,
                    "default": [[1, 0], [0, 1]],
                },
                "layer_v:matrix": {
                    "type": "array",
                    "items": {"type": "array", "items": {"type": "integer"}, "minItems": 2, "maxItems": 2},
                    "minItems": 2,
                    "maxItems": 2,
                    "default": [[1, 0], [0, 1]],
                },
                "distance": {"type": "number", "default": 2.0},
            },
            "required": ["slab_v:matrix", "layer_v:matrix", "distance"],
        },
    },
    "required": ["slab", "interface"],
}


form = Form(inputs_schema)

In [ ]:
# data that comes from JS, here are some examples
poscars = {
    "Ni": """Ni4
1.0
   3.4751458659480110    0.0000000000000000    0.0000000000000002
   0.0000000000000006    3.4751458659480110    0.0000000000000002
   0.0000000000000000    0.0000000000000000    3.4751458659480110
Ni
4
direct
   0.0000000000000000    0.0000000000000000    0.0000000000000000 Ni
   0.0000000000000000    0.5000000000000000    0.5000000000000000 Ni
   0.5000000000000000    0.0000000000000000    0.5000000000000000 Ni
   0.5000000000000000    0.5000000000000000    0.0000000000000000 Ni
""",
    "Cu": """Cu4
1.0
   3.5774306715697510    0.0000000000000000    0.0000000000000002
   0.0000000000000006    3.5774306715697510    0.0000000000000002
   0.0000000000000000    0.0000000000000000    3.5774306715697510
Cu
4
direct
   0.0000000000000000    0.0000000000000000    0.0000000000000000 Cu
   0.0000000000000000    0.5000000000000000    0.5000000000000000 Cu
   0.5000000000000000    0.0000000000000000    0.5000000000000000 Cu
   0.5000000000000000    0.5000000000000000    0.0000000000000000 Cu
""",
    "Au": """Au4
1.0
   4.1712885314747270    0.0000000000000000    0.0000000000000003
   0.0000000000000007    4.1712885314747270    0.0000000000000003
   0.0000000000000000    0.0000000000000000    4.1712885314747270
Au
4
direct
   0.0000000000000000    0.0000000000000000    0.0000000000000000 Au
   0.0000000000000000    0.5000000000000000    0.5000000000000000 Au
   0.5000000000000000    0.0000000000000000    0.5000000000000000 Au
   0.5000000000000000    0.5000000000000000    0.0000000000000000 Au
""",
    "SiC": """Si4 C4
1.0
   4.3539932475828609    0.0000000000000000    0.0000000000000003
   0.0000000000000007    4.3539932475828609    0.0000000000000003
   0.0000000000000000    0.0000000000000000    4.3539932475828609
Si C
4 4
direct
   0.7500000000000000    0.2500000000000000    0.7500000000000000 Si4+
   0.7500000000000000    0.7500000000000000    0.2500000000000000 Si4+
   0.2500000000000000    0.2500000000000000    0.2500000000000000 Si4+
   0.2500000000000000    0.7500000000000000    0.7500000000000000 Si4+
   0.0000000000000000    0.0000000000000000    0.0000000000000000 C4-
   0.0000000000000000    0.5000000000000000    0.5000000000000000 C4-
   0.5000000000000000    0.0000000000000000    0.5000000000000000 C4-
   0.5000000000000000    0.5000000000000000    0.0000000000000000 C4-
""",
    "Graphene": """Graphene
1.0
   2.467291000	   0.000000000	   0.000000000
  -1.233645000	   2.136737000	   0.000000000
   0.000000000	   0.000000000	   7.803074000
C
2
direct
   0.000000000    0.000000000    0.000000000  C
   0.333333000    0.666667000    0.000000000  C
""",
}

In [ ]:
# definition of MaterialInterface class
from ase.build import surface, supercells
from ase.io import read, write
from ase.visualize import view
import io
from operator import itemgetter
import numpy as np

globals()["default_settings"] = {
    "slab": {
        "miller:h": 1,
        "miller:k": 1,
        "miller:l": 1,
        "vacuum": 1,
        "number_of_layers": 3,
    },
    "interface": {"slab_v:matrix": [[1, 0], [0, 1]], "layer_v:matrix": [[1, 0], [0, 1]], "distance": 2.0},
}


def poscar_to_atoms(poscar):
    input = io.StringIO(poscar)
    atoms = read(input, format="vasp")
    return atoms


def write_atoms_to_poscar(atoms):
    output = io.StringIO()
    write(output, atoms, format="vasp")
    content = output.getvalue()
    output.close()
    return content


def poscar_to_cif(poscar):
    input = io.StringIO(poscar)
    atoms = read(input, format="vasp")

    output = io.BytesIO()
    write(output, atoms, format="cif")
    content = output.getvalue()
    output.close()
    return content


def expand_matrix_2x2_to_3x3(matrix_2x2):
    matrix_3x3 = [[0, 0, 0], [0, 0, 0], [0, 0, 1]]

    for i in range(2):
        for j in range(2):
            matrix_3x3[i][j] = matrix_2x2[i][j]

    return matrix_3x3


class MaterialInterface:
    def __init__(self, substrate, material, settings=None):
        self.substrate = substrate
        self.material = material
        self.settings = globals()["default_settings"]
        if settings:
            for key in self.settings.keys():
                if key in settings:
                    self.settings[key].update(settings[key])
        self.structure = self.create_structure()

    def create_structure(self):
        slab = self.settings["slab"]
        interface = self.settings["interface"]

        self.substrate = surface(
            self.substrate,
            (slab["miller:h"], slab["miller:k"], slab["miller:l"]),
            vacuum=slab["vacuum"],
            layers=slab["number_of_layers"],
        )

        slab_v_matrix = expand_matrix_2x2_to_3x3(interface["slab_v:matrix"])
        layer_v_matrix = expand_matrix_2x2_to_3x3(interface["layer_v:matrix"])

        self.substrate = supercells.make_supercell(self.substrate, slab_v_matrix)
        self.substrate.wrap()
        self.material = supercells.make_supercell(self.material, layer_v_matrix)
        self.original_material = self.material.copy()
        self.material.set_cell(self.substrate.get_cell(), scale_atoms=True)
        self.material.wrap()

        z_max_substrate = max(self.substrate.positions[:, 2])
        z_min_material = min(self.material.positions[:, 2])
        z_offset = z_max_substrate - z_min_material + interface["distance"]
        self.material.positions[:, 2] += z_offset

        return self.substrate + self.material

    def calculate_strain(self, substrate=None, material=None):
        """Calculates strain for the material layer on the substrate"""

        if substrate is None:
            substrate = self.substrate
        if material is None:
            material = self.original_material

        substrate_cell = substrate.get_cell()
        material_cell = material.get_cell()

        a0 = np.linalg.norm(substrate_cell[0])
        b0 = np.linalg.norm(substrate_cell[1])

        a1 = np.linalg.norm(material_cell[0])
        b1 = np.linalg.norm(material_cell[1])

        strain_a = (a1 - a0) / a0
        strain_b = (b1 - b0) / b0

        return (strain_a, strain_b)

    def view(self, material=None, repeat=(1, 1, 1)):
        if material is None:
            material = self.structure
        view(material * repeat)

In [ ]:
form.show(width="300px")
print(form.data)

In [ ]:
# actual Pyodide code

global settings, poscar_data
settings = form.data
poscar_data = poscars


def func():
    """This function is a gateway to Pyodide in Materials Designer"""

    poscar_data = globals()["poscar_data"]
    settings = globals()["settings"]
    materials = {}

    for material_name, poscar in poscar_data.items():
        materials[material_name] = poscar_to_atoms(poscar)

    substrate = materials["Au"]
    material = materials["Graphene"]
    interface = MaterialInterface(substrate, material, settings)
    write("structure.poscar", interface.structure, format="vasp")
    print(interface.structure)
    interface.view(repeat=(2, 2, 1))
    print("strain (a, b):", interface.calculate_strain())


func()

In [ ]:
# brute force coincedence lattice search


def get_settings(prefix="slab", max_v=3):
    settings_array = []
    for v1 in range(0, max_v):
        for v2 in range(0, max_v):
            for u1 in range(0, max_v):
                for u2 in range(0, max_v):
                    matrix = [[v1, v2], [u1, u2]]
                    if np.linalg.det(matrix) > 0:
                        settings_array.append({"interface": {f"{prefix}_v:matrix": matrix}})
    return settings_array


def brute_force_optimization(substrate, material):
    strain = []
    settings_slab = get_settings()
    settings_material = get_settings(prefix="layer")
    for settings in settings_slab:
        for settings2 in settings_material:
            settings["interface"].update(settings2["interface"])
            interface = MaterialInterface(substrate, material, settings)
            s = interface.calculate_strain(interface.substrate, interface.original_material)
            strain.append({"strain": s, "settings": settings, "metric": abs(s[0] + s[1])})
    return strain


substrate_for_brute_force = poscar_to_atoms(poscars["Au"])
material_for_brute_force = poscar_to_atoms(poscars["Graphene"])
data = brute_force_optimization(substrate_for_brute_force, material_for_brute_force)

# sort by metric
data = sorted(data, key=itemgetter("metric"))
print(data)

In [ ]:
# Using results of brute force optimization as settings
import matplotlib.pyplot as plt

metric = np.array([d["metric"] for d in data])
plt.plot(metric)
print(data[0]["settings"])
interface = MaterialInterface(substrate_for_brute_force, material_for_brute_force, data[0]["settings"])
interface.view(repeat=(2, 2, 1))